# Putative Human – Mouse BRCA1 Orthologs  
Write a program using NCBI's E-Utilities to retrieve the ids of RefSeq human BRCA1 proteins from NCBI. 
* Use the query: "Homo sapiens"[Organism] AND BRCA1[Gene Name] AND REFSEQ
* Extend your program to search these protein ids (one at a time) vs RefSeq proteins (refseq_protein) using the NCBI blast web-service.
* Further extend your program to filter the results for significance (E-value < 1.0e-5) and to extract mouse sequences (match "Mus musculus" in the description).Note: you may need to 
* Request at least 200 alignments from qblast to see the first mouse protein (keyword parameter hitlist_size, default is 50), or __Restrict the qblast search to mouse refseq proteins (keyword parameter entrez_query)__

Need to use the [Biopython guide](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc91)

Questions
* extend to refseq: search both gb and refseq? or search genes then proteins all in refseq?

In [6]:
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
import os.path

Entrez.email = 'michael.chambers2@nih.gov'

In [ ]:
handle = Entrez.esearch(
    db='protein',
    term = '"Homo sapiens"[Organism] AND BRCA1[Gene Name] AND REFSEQ',
    usehistory='y'    
)

result = Entrez.read(handle)
handle.close()

id_list = ','.join(result['IdList'])
handle = Entrez.efetch(
    db='protein',
    id=id_list,
    rettype='gb'
)

for gi,r in zip(result['IdList'], SeqIO.parse(handle, 'genbank')):
    print(f'\n*** START: {gi} ***\n')
    print('GI:', gi)
    print('Accession:', r.id)
    print('Description:', r.description)

    print(f"\nBLAST for GI {gi}...\n")
    result_handle = NCBIWWW.qblast(
        'blastp',
        'refseq_protein',
        gi,
        expect=1e-5,
        entrez_query='"Mus musculus"[Organism]'
    )
    
    #blast_results = result_handle.read()
    #result_handle.close()
    
    # parse the file (was 38)
    for blast_result in NCBIXML.parse(result_handle):
        for desc in blast_result.descriptions:
            print('***Alignment***')
            print('Sequence:',desc.title)
            print('Evalue:', desc.e)
            print()
    
    file = f'blastp-np-{gi}.xml'    
    save_file = open(file, 'w')
    save_file.write(result_handle.read())
    result_handle.close()
    print(f'*** END {gi} ***')


*** START: 6552299 ***

GI: 6552299
Accession: NP_009225.1
Description: breast cancer type 1 susceptibility protein isoform 1 [Homo sapiens]

BLAST for GI 6552299...

***Alignment***
Sequence: ref|NP_033894.3| breast cancer type 1 susceptibility protein homolog [Mus musculus] >ref|XP_030101355.1| breast cancer type 1 susceptibility protein homolog isoform X1 [Mus musculus]
Evalue: 0.0

***Alignment***
Sequence: ref|XP_006532127.1| breast cancer type 1 susceptibility protein homolog isoform X2 [Mus musculus]
Evalue: 0.0

***Alignment***
Sequence: ref|XP_017169722.1| breast cancer type 1 susceptibility protein homolog isoform X3 [Mus musculus]
Evalue: 0.0

***Alignment***
Sequence: ref|XP_006532131.1| breast cancer type 1 susceptibility protein homolog isoform X4 [Mus musculus]
Evalue: 5.77857e-168

***Alignment***
Sequence: ref|XP_011240255.1| tripartite motif-containing protein 30A-like isoform X3 [Mus musculus]
Evalue: 3.3666e-08

***Alignment***
Sequence: ref|XP_006508443.1| tripart